In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error

SEED = 17

In [2]:
df = pd.read_csv('/content/Housing.csv')

X = df.drop('price', axis=1)
y = df['price']

X_encoded = pd.get_dummies(X, columns=['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=SEED)

model = make_pipeline(
    PolynomialFeatures(degree=2),
    MinMaxScaler(),
    LinearRegression()
)

model.fit(X_train, y_train)

predictions = model.predict(X_test)
mse_sklearn = mean_squared_error(y_test, predictions)
print("Mean Squared Error (MSE) with sklearn:", mse_sklearn)

Mean Squared Error (MSE) with sklearn: 7329068550080376.0


In [3]:
model_sgd = make_pipeline(
    PolynomialFeatures(degree=2),
    MinMaxScaler(),
    SGDRegressor(max_iter=1000, tol=1e-3, random_state=SEED)
)

model_sgd.fit(X_train, y_train)

predictions_sgd = model_sgd.predict(X_test)
mse_sgd = mean_squared_error(y_test, predictions_sgd)
print("Mean Squared Error (MSE) with SGDRegressor:", mse_sgd)

Mean Squared Error (MSE) with SGDRegressor: 866127396611.2178


In [4]:
def hypothesis(X, weights):
    return X.dot(weights)

def loss_function(X, y, weights):
    predictions = hypothesis(X, weights)
    errors = predictions - y
    return np.mean(errors**2)

def gradient_descent_step(X, y, weights, learning_rate):
    predictions = hypothesis(X, weights)
    errors = predictions - y
    gradient = 2/X.shape[0] * X.T.dot(errors)
    weights -= learning_rate * gradient
    return weights

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

def gradient_descent_step_normalized(X, y, weights, learning_rate):
    predictions = hypothesis(X, weights)
    errors = predictions - y
    gradient = 2/X.shape[0] * X.T.dot(errors)
    weights -= learning_rate * gradient
    return weights

X_train_with_bias_scaled = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]
weights_gradient_descent_scaled = np.zeros(X_train_with_bias_scaled.shape[1])

learning_rate = 0.01
for _ in range(1000):
    weights_gradient_descent_scaled = gradient_descent_step_normalized(X_train_with_bias_scaled, y_train.values, weights_gradient_descent_scaled, learning_rate)

print("gradient:", weights_gradient_descent_scaled)

X_train_with_bias = np.c_[np.ones(X_train.shape[0]), X_train]
optimal_weights_analytical = np.linalg.inv(X_train_with_bias.T @ X_train_with_bias) @ X_train_with_bias.T @ y_train.values

print("analitical:", optimal_weights_analytical)

gradient: [ 4.82798508e+06  5.19809271e+05  1.28422745e+05  4.86154254e+05
  3.69002732e+05  2.56722027e+05  1.58417177e+05  1.22338731e+05
  1.51888494e+05  2.25642430e+05  4.52357359e+05  2.64278591e+05
 -1.27071664e+03 -1.93767697e+05]
analitical: [-1.20758660e+05  2.39000824e+02  1.78219428e+05  9.79426744e+05
  4.23734231e+05  2.95971521e+05  4.63103091e+05  3.08791097e+05
  3.18291083e+05  1.05382503e+06  9.74460716e+05  6.16069287e+05
 -2.57226430e+03 -4.14210114e+05]
